# Inverting the Formula for the Ising Coupling Matrix 

## Notes on Network Performance

Currently the parameters $\mu$ and $\Omega$ are taken from a uniform distribution. The output activations will also lie in the (0,1) interval. The expected distance between two values drawn from a uniform [0,1] distribution is $\frac{1}{3}$. Since we have $n^2+n$ parameters for an ion chain of size n, we get that if the network is outputting just a guess that the cost should be approximately $\frac{n^2+n}{6}$.(since we are currently using a quadratic cost function)

For a network with n=3 I have been able to get a cost of just under a quarter of this value for my synthetic data set up to this point.
I am still researching ways to improve hyperparameter selection as well as improved network architecture.

One question I have is to evaluate what is a good cost value for the test set and what is not. And whether or not it is a good physical representation of the system to have the parameters taken from a uniform distribution.

In [ ]:
import ionchain
import classes
import pickle
import numpy as np
import network
import network2
% matplotlib inline

## Functions to Model the Physical System
These functions, Lambe_Dicke J_ij1 and J_ij used to make the necessary calculations to model an instance of an ion chain.

In [ ]:
def Lamb_Dicke(ic):
    """Computes Lamb-Dicke parameters for a trapped ion chain 
    
    Args:
        ic (Object) - an instance of a trapped ion chain
        
    Returns:
        An nxn matrix where n is the number of ions in the ion chain.
        The i,mth entry of the matrix corresponds to the Lambe-Dicke
        parameter eta[i,m] which sets the scale for the coupling between
        spin i and mode m.
    """
    deltak = 2*1.7699*10**7
    M = 2.8395 * 10**(-25)
    hbar = 1.0546e-34
    n = ic.n
    b_ij = ic.x_eigvecs
    omega_m = ic.x_freqs
    eta = np.empty([n,n])
    for m in range(n):
        for i in range(n):
            eta[i,m] = b_ij[i,m]*deltak*((hbar/(2*M*omega_m[m]))**0.5)
    return eta

In [ ]:
def J_ij1(mu, Rabi_freq, ic):
    """Computes spin-spin coupling beteen atoms in a trapped ion
    chain with global beatnote detuning. 
    
    Args:
        mu(float) - global beatnote detuning parameter
        ic (Object) - an instance of a trapped ion chain
        Rabi_freq(An 1Xn vector of floats, where n is number of ions
        in the chain) 
            - containing where the ith entry is the single spin Rabi
            frequency of atom i
        
    Returns:
        An nxn matrix where n is the number of ions in the ion chain.
        The i,jth is the spin-spin coupling beteen atoms i and j in the
        ion chain.
    """
    n = ic.n
    eta = Lamb_Dicke(ic)
    omega_m = ic.x_freqs
    J = np.empty((n,n))
    for i in range(n):
        J[i,i] = 0
        for j in range(i+1,n):
            J[i,j] = Rabi_freq[i]*Rabi_freq[j]* np.sum(eta[i, :] * eta[j, :] * omega_m[:] /(np.full((1,n),mu)**2 - omega_m[:]**2))
    return J

In [ ]:
def J_ij(mus, Rabi_freq_matrix, ic):
    n = ic.n
    eta = Lamb_Dicke(ic)
    omega_m = ic.x_freqs
    F = np.empty((n, n, len(mus)))
    for i in range(n):
        for j in range(n):
            for m in range(len(mus)):
                F[i, j, m] = np.sum(eta[i, :] * eta[j, :] * omega_m[:] /
                                    (mus[m]**2 - omega_m[:]**2))
    J = np.empty((n, n))
    for i in range(n):
        J[i, i] = 0
        for j in range(i + 1, n):
            J[i, j] = np.sum(Rabi_freq_matrix[i, :] * Rabi_freq_matrix[j, :] * F[i, j, :])
            J[j, i] = J[i, j]
    return J

## Generating Synthetic Data
The functions test_data and create_data_file are used to generate synthetic data for training, testing, and validation.

To create a synthetic data set one must just modify the code in the cell below the one where these functions are defined.

In [ ]:
def test_data(ic, data_size):
    """Generates training, test and validation data for a 
    particular ion chain ic. The function returns 3 tuples corresponding 
    to each data set. Each tuple contains 2 items. The first is the J_ij
    values for the system in vectorized form(by row). If n is the size of
    the ion chain, the second contains a vector whose first n entires
    are the parameters mu and the remaining entires are the vectorized
    form of the Rabi frequency matrix
    
    Arguments:
    ic(Object from IonChain Class)-the ion chain from which to construct
    data from
    data_size(Int)-desired data size"""
    n = ic.n
    np.random.seed(123)
    
    # Generate random values for the Rabi matricies and detunings(the first mu_size values correspond
    #to mu and the rest to the Rabi frequencies.
    # The size of the ion chain will determine the number of output values in the network.
    data = []
    for i in range (0, data_size):
        mu = np.random.uniform(low=0, high=1, size=(n,1))
        Rabi = np.random.uniform(low=0, high=1, size=(n,n))     
    # For each set of parameters y we generate corresponding Ising coupling matrix J which
    # will be our input parameters for the neural network
        J = J_ij(mu, Rabi, ic)
        x = np.reshape(J,(n**2,1))
        y = np.concatenate((mu, np.reshape(Rabi,(n**2,1))), axis=0)
        data.append((x,y))  
    
    # Return the dataset in pairs of x,y for train/test/validation sets.
    return data

def create_data_file(filename, ic, data_size):
    """Creates a pickled file containing synthetic test data.
    Arguments:
    filename(string)-desired filename
    ic(Object from IonChain Class)-the ion chain from which to construct
    data from
    data_size(Int)-desired data size"""
    data = test_data(ic, data_size)
    f = open(filename, 'wb')
    pickle.dump(data, f)
    f.close()
    

In [ ]:
ionchain3 = ionchain.IonChain(3,[5,1])
#create_data_file('chain_len3.pickle',ionchain3,100000)

## Training and Validation
The following code is used to define a create a neural network and its architecture, using a modified version of Micheal Nielsons network2.py file, then train it. After running this code the output will currently be the cost for the training and test data at the end of each epoch stored in an array. Along with a plot for visualization.


In [ ]:
net = network2.Network([9,100,12])
data = pickle.load(open('chain_len3.pickle', "rb"))
# train_data = data[:int(len(data)*0.8)]
#test_data = data[int(len(data)*0.8):]
#train_data = data[:1000]
#test_data = data[1000:1100]
#net.SGD(train_data, 1000, 10, 0.001,lmbda = 0, evaluation_data=test_data,eta_update=0)


## Testing
The following code can be modified and used for testing.

In [ ]:
#net.save('Network_with_cost_0.4_for_n=3.txt')

In [ ]:
net1 = network2.load('Network_with_cost_0.4_for_n=3.txt')

In [ ]:
net1.total_cost_no_reg(data)